<a href="https://colab.research.google.com/github/thejatinbabbar/spam-detection/blob/main/new_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

from datetime import datetime
from sklearn.cluster import SpectralClustering
import seaborn as sns
import networkx
import matplotlib.pyplot as plt

In [3]:
DATA_PATH = '/content/drive/My Drive/Data_Mining'

In [4]:
weight = pd.read_csv(f'{DATA_PATH}/Social_spammers_dataset/graphs/app_based_similarity/app_user_interaction_graph/weighted_app_user_edges.csv')

usr_ids = pd.read_csv(f'{DATA_PATH}/Social_spammers_dataset/users/coded_ids.csv')

**Creating Number of applications per user as a new feature**

In [5]:
number_of_apps_for_each_user_df=weight.pivot_table(index=['user_id'], aggfunc='size').reset_index(name='number_of_apps_for_each_user')
number_of_apps_for_each_user_df=pd.merge(usr_ids, number_of_apps_for_each_user_df, on='user_id').drop(['user_id'], axis=1)
number_of_apps_for_each_user_df

,coded_id,number_of_apps_for_each_user
0,1,2
1,2,3
2,3,2
3,4,5
4,5,2
...,...,...
762,763,3
763,764,2
764,765,7
765,766,4


**Creating clusters based on Similarity Matrix and adding it as a new feature**

In [6]:
sim = []
with open(f'{DATA_PATH}/Social_spammers_dataset/graphs/app_based_similarity/sim_matrix.txt') as my_file:
    for line in my_file:
        
        letter_list = line.rstrip().split(",")
        letter_list=[float(i) for i in letter_list]
        sim.append(letter_list)

In [7]:
mat = np.matrix(sim)
clusters=SpectralClustering(12).fit_predict(mat)

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/_spectral.py:482: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"


In [8]:
users_with_clusters = usr_ids
users_with_clusters['clusters']=clusters
users_with_clusters=users_with_clusters.drop(['user_id'], axis=1)
users_with_clusters

,coded_id,clusters
0,1,11
1,2,4
2,3,4
3,4,11
4,5,3
...,...,...
762,763,4
763,764,4
764,765,4
765,766,4


**New two features**

In [11]:
new_features=pd.merge(number_of_apps_for_each_user_df, users_with_clusters, on='coded_id')
new_features

,coded_id,number_of_apps_for_each_user,clusters
0,1,2,11
1,2,3,4
2,3,2,4
3,4,5,11
4,5,2,3
...,...,...,...
762,763,3,4
763,764,2,4
764,765,7,4
765,766,4,4
